In [ ]:
import pandas as pd
import os
from ast import literal_eval

In [ ]:
songs_composition = pd.read_csv('data/songs_composition.csv')

In [ ]:
def df_get_id(filenames):
    ids = []
    for filename in filenames:
        id_ = filename[0].split('/')[-1].split('.')[0]
        ids.append(id_)
    return ids

def df_get_topics(df_topics):
    list_topics = []
    for row in df_topics:
        topics = {}
        for i in range(20):
            topics[i] = row[i + 1]
        list_topics.append(topics)
    return list_topics

In [ ]:
songs_composition['track_id'] = df_get_id(songs_composition.values)
songs_composition['topics'] = df_get_topics(songs_composition.values)
data_songs = pd.read_csv('data/data_lyrics.csv')

In [ ]:
id_topics = songs_composition[['track_id', 'topics']]
result = pd.merge(id_topics, data_songs, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'))

genres = result['genres'].values
genres = set(genres)

topics_genres = {}
topics_all = {}


for index, row in result.iterrows():
    genre = row['genres']
    topics = row['topics']
    year = row['year']
    
    if genre not in  topics_genres:
        topics_genres[genre] = {}
    
    for topic in topics:
        if not topic in topics_genres[genre]:
            topics_genres[genre][topic] = {}
        if not year in topics_genres[genre][topic]:
            topics_genres[genre][topic][year] = []
            topics_genres[genre][topic][year].append(0)
            topics_genres[genre][topic][year].append(0)
        topics_genres[genre][topic][year][0] += topics[topic]
        topics_genres[genre][topic][year][1] += 1
        
        if topic not in topics_all:
            topics_all[topic] = {}
        
        if not year in topics_all[topic]:
            topics_all[topic][year] = []
            topics_all[topic][year].append(0)
            topics_all[topic][year].append(0)
        topics_all[topic][year][0] += topics[topic]
        topics_all[topic][year][1] += 1

Now that we have the corresponding topics coefficients for a given song id, we add them to the all dataframe.

In [ ]:
results = {}
for genre in topics_genres:
    results[genre] = {}
    for topic in topics_genres[genre]:
        results[genre][topic] = {}
        for year in topics_genres[genre][topic]:
            if year:
                mean = topics_genres[genre][topic][year][0] / topics_genres[genre][topic][year][1] 
                results[genre][topic][year] = mean
results

results_all = {}
for topic in topics_all:
    results_all[topic] = {}
    for year in topics_all[topic]:
        if year:
            mean = topics_all[topic][year][0] / topics_all[topic][year][1] 
            results_all[topic][year] = mean

In [ ]:
path = 'data/final_data/topics'
for genre in results:
    for topic in results[genre]:
        directory = path + '/' + genre + '/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        with open((directory + str(topic) + '.csv'), 'w') as csvfile:
            for year in sorted(results[genre][topic].keys()):
                csvfile.write(str(year) + ', ' + str(results[genre][topic][year]) + '\n')

path = 'data/final_data/topics'
directory = path + '/' + 'all' + '/'
if not os.path.exists(directory):
    os.makedirs(directory)
    
for topic in results_all:
    with open((directory + str(topic) + '.csv'), 'w') as csvfile:
        print('year,value', file=csvfile)
        for year in sorted(results_all[topic].keys()):
            csvfile.write(str(year) + ', ' + str(results_all[topic][year]) + '\n')